# Book Example: econ
[![econ.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/book/econ.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/book/econ.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/book/econ.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/book/econ.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/book/econ.ipynb)

Description: book example autogenerated using econ.mod, econ.dat, and econ.run

Tags: ampl-only, ampl-book

Notebook author: N/A

Model author: N/A


In [ ]:
# Install dependencies
!pip install -q amplpy ampltools

In [ ]:
# Google Colab & Kaggle interagration
MODULES=['ampl', 'coin', 'cplex']
from ampltools import cloud_platform_name, ampl_notebook
from amplpy import AMPL, register_magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = ampl_notebook(modules=MODULES) # Install AMPL and use it
register_magics(ampl_object=ampl) # Evaluate %%ampl_eval cells with ampl.eval()

### Example: econ
autogenerated using econ.mod, econ.dat, and econ.run


In [ ]:
%%writefile econ.mod
set PROD;   # products
set ACT;    # activities

param cost {ACT} > 0;      # cost per unit of each activity
param demand {PROD} >= 0;  # units of demand for each product
param io {PROD,ACT} >= 0;  # units of each product from
                           # 1 unit of each activity

param level_min {ACT} > 0; # min allowed level for each activity
param level_max {ACT} > 0; # max allowed level for each activity

var Price {i in PROD};
var Level {j in ACT};

minimize Total_Cost:  sum {j in ACT} cost[j] * Level[j];

subject to Pri_Compl {i in PROD}:
   Price[i] >= 0 complements
      sum {j in ACT} io[i,j] * Level[j] >= demand[i];

subject to Lev_Compl {j in ACT}:
   level_min[j] <= Level[j] <= level_max[j] complements
      cost[j] - sum {i in PROD} Price[i] * io[i,j];
      



In [ ]:
%%writefile econ.dat
data;

param: ACT:   cost  level_min  level_max :=
        P1    2450     240        1000
        P1a   1290     270        1000
        P2    1850     220        1000
        P2a   3700     260        1000
        P2b   2150     200        1000
        P3    2200     260        1000
        P3c   2370     220        1000
        P4    2170     240        1000 ;

param: PROD:  demand :=
       AA1     70000 
       AC1     80000
       BC1     90000 
       BC2     70000 
       NA2    400000 
       NA3    800000  ;

param io (tr):
         AA1  AC1  BC1  BC2   NA2   NA3 :=
   P1     60   20   10   15   938   295
   P1a     8    0   20   20  1180   770
   P2      8   10   15   10   945   440
   P2a    40   40   35   10   278   430
   P2b    15   35   15   15  1182   315
   P3     70   30   15   15   896   400
   P3c    25   40   30   30  1029   370
   P4     60   20   15   10  1397   450 ;



In [ ]:
%%ampl_eval
model econ.mod;
data econ.dat;
option solver cplex;
solve;

